# **QuipuSpicyCycle.**
In this lesson you will:
- Сalculate the rate of TezQuipu pair buy on the exchange QuipuSwap exchange
- Сalculate the rate of TezQuipu pair buy on the exchange SpicySwap exchange
- Сalculate the possible profit from arbitration

As usual, let's start by installing the libraries

In [ ]:
%%capture
## %%capture is a Colab built-in function to suppress the output of that particular cell whether it uses a command-line code or some python code

from matplotlib import pyplot as plt
from pandas import json_normalize
import json
import pandas as pd
from tqdm import tqdm
import requests
import time

We considered the contract from the QuipuSwap in one of the previous lessons, let's see the contract from the SpicySwap

In [ ]:
smart_contract ="KT1VLvdV1u268eVzBFEFr72JgEKhq2MU6NqJ"


request_url = f'https://api.tzkt.io/v1/contracts/{smart_contract}/storage/'

response = requests.get(request_url)
new_data = json.loads(response.text)
new_data

{'assets': {'ledger': 19755,
  'operators': 19756,
  'token_metadata': 19757,
  'token_total_supply': 19758},
 'contract_code': 19759,
 'current_swap': None,
 'farm_proxy_contract': 'KT1QAPkX4XzZp5GagvTuFmUvUGkEXbbfvS4Y',
 'metadata': 19760,
 'price_cumulative_0': '6011597',
 'price_cumulative_1': '21957752',
 'price_ts_last': '2022-02-20T15:50:00Z',
 'reserve0': '2771557423',
 'reserve1': '1550047889',
 'spiceFeeLastK': '4295737320316275140',
 'token0': {'fa2_address': 'KT193D4vozYnhGJQVtw7CoxxqphqUEEwK6Vb',
  'token_id': '0'},
 'token1': {'fa2_address': 'KT1PnUZCp3u2KzWr93pn4DD7HAJnm3rWVrgn',
  'token_id': '0'}}

As you can see the storage is different
reserve0 - Quipu pool
reserve1- Tezos pool

# **Collecting reserves**

Let's write functions that get pools-reserves

In [ ]:
def Qpools(smart_contract):
    request_url = f'https://api.tzkt.io/v1/contracts/{smart_contract}/storage/'

    response = requests.get(request_url)
    new_data = json.loads(response.text)
    return int(new_data['storage']['tez_pool']), int(new_data['storage']['token_pool'])

def Spools(mart_contract):
    request_url = f'https://api.tzkt.io/v1/contracts/{smart_contract}/storage/'

    response = requests.get(request_url)
    new_data = json.loads(response.text)
    return int(new_data['reserve1']),int(new_data['reserve0'])

Let's take the functions from the previous lessons to calculate the exchange rate of the pair

In [ ]:
#1 tez = 1,000,000 micro tez
def tzToMutez(tezAmount):
    return tezAmount*1000000
    
def TeztoQoutput(tezAmount,tez_pool,token_pool):
    mutezAmount = tzToMutez(tezAmount)
    
    tezInWithFee = mutezAmount*997
    
    numerator = tezInWithFee*token_pool
    
    denominator = tez_pool * 1000 + tezInWithFee
    
    tokensOut = numerator/denominator

    return tokensOut/(10*10*10*10*10*10)

Let's calculate the rate

In [ ]:
def ex_rate_tq(amount):
    return (amount)/TeztoQoutput(amount)

def pair(qcontract,scontract,tezAmount):
    q_tez_pool,q_token_pool = Qpools(qcontract)
    q=TeztoQoutput(tezAmount,q_tez_pool,q_token_pool)
    print("Quipu_rate: ",q)
    s_tez_pool,s_token_pool = Spools(scontract)
    s=TeztoQoutput(tezAmount,s_tez_pool,s_token_pool)
    print("Spicy_rate: ",s)

Let's add the calculation of the reverse rate on the exchange and see the possible profit

In [ ]:
def pair(qcontract,scontract,tezAmount):
    q_tez_pool,q_token_pool = Qpools(qcontract)
    q=TeztoQoutput(tezAmount,q_tez_pool,q_token_pool)
    print("Quipu_rate: ",q)
    s_tez_pool,s_token_pool = Spools(scontract)
    s=TeztoQoutput(tezAmount,s_tez_pool,s_token_pool)
    print("Spicy_rate: ",s)
    #Exchange rate
    reverseSpicy = tezAmount/s
    print("Spicy reverse: ", reverseSpicy)
    print(tezAmount - tezAmount*q*reverseSpicy)



# Important!!! 
Transactional fees are not included

In [ ]:
qcontract="KT1X3zxdTzPB9DgVzA3ad6dgZe9JEamoaeRy"
scontract="KT1VLvdV1u268eVzBFEFr72JgEKhq2MU6NqJ"
tezAmount= 1
pair(qcontract,scontract,tezAmount)

Quipu_rate:  1.7600589249150516
Spicy_rate:  1.7815362858469792
Spicy reverse:  0.5613132934446964
0.01205552819920086


## Task1 
Since transactional fees are not included your goal will be extend the pair function so that it also takes into account transaction costs